In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None,784],dtype=tf.float32)
    labels = tf.placeholder(shape=[None,10],dtype=tf.float32)
    
    hidden_weight = tf.Variable(tf.random_normal([784,128]),name='hidden_weight')
    hidden_bias = tf.Variable(tf.zeros([128,]),name='hidden_bias')
    
    hidden_output = tf.nn.relu(tf.matmul(inputs,hidden_weight) + hidden_bias)
    
    output_weight = tf.Variable(tf.random_normal([128,10]),name='output_weight')
    output_bias = tf.Variable(tf.zeros([10,]),name='output_bias')
    
    logits = tf.matmul(hidden_output,output_weight) + output_bias
    output = tf.nn.softmax(logits)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=labels))
    
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(labels,axis=1),tf.argmax(output,axis=1)),tf.float32))

In [7]:
with tf.Session(graph=g) as sess:
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss,_ = sess.run([loss,train_op],feed_dict={
            inputs:batch_images,
            labels:batch_labels
        })
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images,batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc,feed_dict={
                    inputs:batch_images,
                    labels:batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 114.1995, acc 0.2024
step   500, loss 3.0882, acc 0.7631
step  1000, loss 1.0618, acc 0.8075
step  1500, loss 6.2814, acc 0.8305
step  2000, loss 3.0803, acc 0.8397
step  2500, loss 4.5529, acc 0.8536
step  3000, loss 0.1493, acc 0.8555
step  3500, loss 5.4211, acc 0.8659
step  4000, loss 2.9003, acc 0.8653
step  4500, loss 2.1427, acc 0.8733
step  5000, loss 1.4414, acc 0.8719
step  5500, loss 1.6811, acc 0.8734
step  6000, loss 1.1963, acc 0.8767
step  6500, loss 1.5536, acc 0.8770
step  7000, loss 0.8804, acc 0.8803
step  7500, loss 1.6436, acc 0.8803
step  8000, loss 0.9773, acc 0.8837
step  8500, loss 0.8565, acc 0.8857
step  9000, loss 3.4974, acc 0.8837
step  9500, loss 0.6003, acc 0.8891
step 10000, loss 0.3092, acc 0.8871
step 10500, loss 0.1238, acc 0.8903
step 11000, loss 0.2798, acc 0.8904
step 11500, loss 0.3829, acc 0.8875
step 12000, loss 1.8498, acc 0.8900
step 12500, loss 2.0709, acc 0.8862
step 13000, loss 2.2715, acc 0.8883
step 13500, loss 0.7257, a

使用高阶API构建

In [13]:
with tf.Graph().as_default() as g1:
    inputs = tf.placeholder(shape=[None,784],dtype=tf.float32)
    labels = tf.placeholder(shape=[None,10],dtype=tf.float32)
    
    hidden_output = tf.keras.layers.Dense(128,activation=tf.nn.relu)(inputs)
    logits = tf.keras.layers.Dense(10,activation=None)(hidden_output)
    
    output = tf.nn.softmax(logits)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits,labels=labels))
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(labels,axis=1),tf.argmax(output,axis=1)),
                                tf.float32))
    

In [14]:
with tf.Session(graph=g1) as sess:
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))


step     0, loss 2.3885, acc 0.0815
step   500, loss 0.6244, acc 0.8549
step  1000, loss 0.6903, acc 0.8848
step  1500, loss 0.4401, acc 0.8974
step  2000, loss 0.6157, acc 0.9075
step  2500, loss 0.2208, acc 0.9088
step  3000, loss 0.2388, acc 0.9141
step  3500, loss 0.4034, acc 0.9185
step  4000, loss 0.3893, acc 0.9219
step  4500, loss 0.2438, acc 0.9233
step  5000, loss 0.1616, acc 0.9252
step  5500, loss 0.2773, acc 0.9283
step  6000, loss 0.1970, acc 0.9302
step  6500, loss 0.0639, acc 0.9298
step  7000, loss 0.3346, acc 0.9342
step  7500, loss 0.3655, acc 0.9341
step  8000, loss 0.1904, acc 0.9348
step  8500, loss 0.1393, acc 0.9350
step  9000, loss 0.1413, acc 0.9370
step  9500, loss 0.3052, acc 0.9383
step 10000, loss 0.1346, acc 0.9407
step 10500, loss 0.5041, acc 0.9403
step 11000, loss 0.0985, acc 0.9437
step 11500, loss 0.1787, acc 0.9432
step 12000, loss 0.2274, acc 0.9424
step 12500, loss 0.2433, acc 0.9473
step 13000, loss 0.2484, acc 0.9439
step 13500, loss 0.1602, acc

In [ ]:
#使用高阶API使得模型的效率变高了，代码简洁方便